In [1]:
!pip -q install tensorboard tensorboard_plugin_profile tensorflow_datasets

In [2]:
from datetime import datetime
from packaging import version

import os
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)

ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])

2023-01-17 14:20:20.859276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 14:20:21.044107: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version:  2.10.0


2023-01-17 14:20:23.614884: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-17 14:20:23.614932: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-12-144.eu-west-2.compute.internal): /proc/driver/nvidia/version does not exist
2023-01-17 14:20:23.615781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 14:20:23.833420: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-01-17 14:20:23.833458: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


Epoch 1/2


2023-01-17 14:20:23.999574: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


469/469 [==============================] - 3s 5ms/step - loss: 0.3596 - accuracy: 0.9007 - val_loss: 0.1864 - val_accuracy: 0.9458
Epoch 2/2
 45/469 [=>............................] - ETA: 0s - loss: 0.1923 - accuracy: 0.9444

2023-01-17 14:20:27.109082: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-01-17 14:20:27.109120: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-01-17 14:20:27.166415: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2023-01-17 14:20:27.252986: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


 75/469 [===>..........................] - ETA: 2s - loss: 0.1891 - accuracy: 0.9467

2023-01-17 14:20:27.377866: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20230117-142023/plugins/profile/2023_01_17_14_20_27

2023-01-17 14:20:27.471719: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/20230117-142023/plugins/profile/2023_01_17_14_20_27/ip-172-16-12-144.eu-west-2.compute.internal.trace.json.gz
2023-01-17 14:20:27.512032: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20230117-142023/plugins/profile/2023_01_17_14_20_27

2023-01-17 14:20:27.512206: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/20230117-142023/plugins/profile/2023_01_17_14_20_27/ip-172-16-12-144.eu-west-2.compute.internal.memory_profile.json.gz
2023-01-17 14:20:27.513655: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/20230117-142023/plugins/profile/2023_01_1

469/469 [==============================] - 2s 3ms/step - loss: 0.1614 - accuracy: 0.9540 - val_loss: 0.1354 - val_accuracy: 0.9609


In [3]:
# Load the TensorBoard notebook extension.
%reload_ext tensorboard
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs
